<a href="https://colab.research.google.com/github/ailab-nda/ML/blob/main/chap06.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 第6章

教科書で取り上げた boston データは差別的な属性が含まれており、倫理的な問題があるとして scikit-learn ver1.2で削除されました。

回帰用の toy data としては、代わりに [California housing dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.fetch_california_housing.html) を使うことが推奨されています。

California housing データはブロックグループ内の収入の中央値、平均部屋数、平均世帯人数、緯度、経度などと不動産価格の関係を示したものです。このデータを用いて線形回帰と正則化、回帰木について学びます。


In [ ]:
#!pip install scikit-learn --trusted-host pypi.python.org --trusted-host files.pythonhosted.org --trusted-host pypi.org

In [ ]:
#出力を小数点以下3桁に制限。DataFrame内では設定は無効
%precision 3

## 線形回帰

必要なライブラリを読み込みます。

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score

### データの読み込み

dataをX、targetをyに格納した後、DESCR属性を表示します。

In [ ]:
from sklearn.datasets import fetch_california_housing
housing = fetch_california_housing()
X = housing.data
y = housing.target
print(housing.DESCR)

np.set_printoptions(suppress=True)  として、ndarrayのデータを表示させたときに指数表示を行わないようにしてから、学習データXの冒頭5件ほどの内容を確認します。

In [ ]:
np.set_printoptions(suppress=True)
X[0:5]

In [ ]:
y[0:5]

特徴のスケールがかなり異なるので、標準化しておきます。

In [ ]:
ss = StandardScaler()
X = ss.fit_transform(X)

組み込みデータセットの特徴名はfeature_names属性の値として文字列配列の形式で得られます。回帰式の解釈に必要になるので確認しておきます。

In [ ]:
housing.feature_names

[LinearRegression](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html)で線形回帰関数の学習を行います。

In [ ]:
lr = LinearRegression()
lr.fit(X, y)

係数と係数の二乗和を表示します。

In [ ]:
for f, w in zip(housing.feature_names, lr.coef_) :
    print(f"{f:7s}: {w:6.2f}")

In [ ]:
sum(lr.coef_**2) #係数の二乗和

### 評価

交差確認による[決定係数](http://mathtrain.jp/ketteikeisu)を用いた評価を行います。評価法はcross_val_scoreメソッドのscoring引数で決定係数を指定します。scoring引数の可能な値は[こちら](http://scikit-learn.org/stable/modules/model_evaluation.html)。また、交差確認における分割をランダムにするため、[ShuffleSplit](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html)を使います。


In [ ]:
cv = ShuffleSplit(n_splits=10)
scores = cross_val_score(lr, X, y, cv=cv, scoring='r2')
print(f"{scores.mean():.2f} (+/- {scores.std():.2f})")

## 正則化

[Ridge回帰(L2)](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)
のパラメータalphaの値でL2正則化項の重みを調整することで、汎化性能が上がる可能性があります。

In [ ]:
lr2 = Ridge(alpha=10.0)
lr2.fit(X, y)

係数と係数の二乗和を表示します。

In [ ]:
for f, w in zip(housing.feature_names, lr2.coef_) :
    print(f"{f:7s}: {w:6.2f}")

In [ ]:
sum(lr2.coef_**2)

交差確認による決定係数を用いた評価

In [ ]:
scores = cross_val_score(lr2, X, y, cv=cv, scoring='r2')
print(f"{scores.mean():.2f} (+/- {scores.std():.2f})")

[Lasso回帰(L1)](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
のパラメータalphaの値を大きくすれば、L1正則化項の重みが大きくなり、重みが0となる次元が増えます。

In [ ]:
lr3 = Lasso(alpha=0.05)
lr3.fit(X, y)

係数と係数の二乗和を表示します。

In [ ]:
for f, w in zip(housing.feature_names, lr3.coef_) :
    print(f"{f:7s}: {w:6.2f}")

In [ ]:
sum(lr3.coef_**2)

結果に影響の大きい特徴をリストアップ

In [ ]:
np.array(housing.feature_names)[abs(lr3.coef_) > 0.1]

交差確認による決定係数を用いた評価

In [ ]:
scores = cross_val_score(lr3, X, y, cv=cv, scoring='r2')
print(f"{scores.mean():.2f} (+/- {scores.std():.2f})")

## 回帰木

回帰木の作成。深さは3にしておきます。

In [ ]:
rt = DecisionTreeRegressor(max_depth = 3)
rt.fit(X, y)

木の表示を行います。ノードの色の濃さはそのノードに割り当てられた出力値の平均を反映しています。

In [ ]:
plt.figure(figsize=(15,12))
plot_tree(rt, filled=True, feature_names=housing.feature_names, fontsize=10)
plt.show()

### 評価

交差確認による決定係数を用いた評価

In [ ]:
scores = cross_val_score(rt, X, y, cv=cv, scoring='r2')
print(f"{scores.mean():.2f} (+/- {scores.std():.2f})")

### ハイパーパラメータの変更

木を浅くしてみます。

In [ ]:
rt2 = DecisionTreeRegressor(max_depth = 2)
rt2.fit(X,y)

木の表示

In [ ]:
plt.figure(figsize=(15,12))
plot_tree(rt2, filled=True, feature_names=housing.feature_names, fontsize=10)
plt.show()

交差確認による決定係数を用いた評価

In [ ]:
scores = cross_val_score(rt2, X, y, cv=cv, scoring='r2')
print(f"{scores.mean():.2f} (+/- {scores.std():.2f})")

木を深くしてみます。

In [ ]:
rt3 = DecisionTreeRegressor(max_depth = 6)
rt3.fit(X,y)

木の表示

In [ ]:
plt.figure(figsize=(15,12))
plot_tree(rt3, filled=True, feature_names=housing.feature_names, fontsize=10)
plt.show()

交差確認による決定係数を用いた評価

In [ ]:
scores = cross_val_score(rt3, X, y, cv=cv, scoring='r2')
print(f"{scores.mean():.2f} (+/- {scores.std():.2f})")

二乗誤差の値を見ると、深い木の方が性能が高くなっています。これはこのデータのみに言えることで、一般化はできません。

# 演習問題

scikit-learn付属のdiabetesデータに対して、線形回帰を行ってください。また、正則化（L1,L2)の効果を測定してください。

### ライブラリの読み込み

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score

### データの読み込み

In [ ]:
diabetes = load_diabetes(as_frame=True)
print(diabetes.DESCR)
X = diabetes.data
y = diabetes.target

In [ ]:
X

組み込みデータセットの特徴名はfeature_names属性の値として文字列配列の形式で得られます。回帰式の解釈に必要になるので確認しておきます。

In [ ]:
diabetes.feature_names

### 評価

## 正則化